# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Convert Max Temp from Kelvin to Celsius.
city_data_df["Max Temp"] = city_data_df["Max Temp"] - 273.1

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.04,73,75,6.69,CC,1689869089
1,1,ilulissat,69.2167,-51.1000,10.06,87,75,3.09,GL,1689869132
2,2,ovalle,-30.5983,-71.2003,21.59,33,100,0.78,CL,1689869020
3,3,gadzhiyevo,69.2551,33.3362,10.25,88,100,9.33,RU,1689868907
4,4,revda,56.8010,59.9303,16.08,82,37,3.45,RU,1689869175


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles="OSM",
                                    frame_width = 600, frame_height = 400, alpha=0.5,
                                    size = 'Humidity', scale = 0.7, color = 'City')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
        (city_data_df["Max Temp"] >= 21) &
        (city_data_df["Max Temp"] <= 35) &
        (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,newman,37.3138,-121.0208,26.24,24,0,2.48,US,1689869111
9,9,dondo,-19.6094,34.7431,23.78,75,7,3.25,MZ,1689869176
25,25,ribeira grande,38.5167,-28.7000,23.27,88,20,2.57,PT,1689869119
26,26,puerto quijarro,-17.7833,-57.7667,33.60,30,2,2.32,BO,1689869180
27,27,macapa,0.0389,-51.0664,32.04,66,40,5.66,BR,1689869180


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()[["City", "Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
7,newman,US,37.3138,-121.0208,24,
9,dondo,MZ,-19.6094,34.7431,75,
25,ribeira grande,PT,38.5167,-28.7000,88,
26,puerto quijarro,BO,-17.7833,-57.7667,30,
27,macapa,BR,0.0389,-51.0664,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
dondo - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
puerto quijarro - nearest hotel: No hotel found
macapa - nearest hotel: Equatorial Hotel
port mathurin - nearest hotel: Escale Vacances
fale old settlement - nearest hotel: No hotel found
puerto colombia - nearest hotel: Hotel Puerto Colombia
albany - nearest hotel: No hotel found
calama - nearest hotel: Apart Hotel Geotel Calama
kingston - nearest hotel: Aphrodite Hotel
the pas - nearest hotel: Wescana Inn
purpe - nearest hotel: No hotel found
misratah - nearest hotel: الفندق السياحى
amuntai - nearest hotel: Grand Balqis Hotel
shimoda - nearest hotel: ホテルウラガ
brookings - nearest hotel: Quality Inn
eregli - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
hithadhoo - nearest hotel: Pebbles Inn
saipan - nearest hotel: Chalan Kanoa Beach Hotel
georgetown - nearest hotel: Page 63 hostel
axim - nearest hotel: Axim Beach Hote

,City,Country,Lat,Lng,Humidity,Hotel Name
7,newman,US,37.3138,-121.0208,24,No hotel found
9,dondo,MZ,-19.6094,34.7431,75,No hotel found
25,ribeira grande,PT,38.5167,-28.7000,88,Hospedaria JSF
26,puerto quijarro,BO,-17.7833,-57.7667,30,No hotel found
27,macapa,BR,0.0389,-51.0664,66,Equatorial Hotel
...,...,...,...,...,...,...
578,san jose del cabo,MX,23.0589,-109.6977,59,Hotel Posada Terranova
581,turkmenbasy,TM,40.0222,52.9552,41,Türkmenbaşy Oteli
582,chulakkurgan,KZ,43.7650,69.1814,21,адеми
589,nicoya,CR,10.1483,-85.4520,67,Hotel marianela


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles="OSM",
                                    frame_width = 600, frame_height = 400, alpha=0.5,
                                    size = 'Humidity', scale = 0.7, color = 'City',
                                    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)